In [1]:
!pip install pandas openpyxl

**Step 1**

In [4]:
# Convert all the date values in the mentioned column to "-"

import pandas as pd
import re
import datetime

# Load the Excel file
file_path = 'Feb.csv'
output_file = 'cleaned_data_1.xlsx'

# Read the Excel file
df = pd.read_csv(file_path)

# Define a pattern to match date formats like '1900-01-01 01:40:00' or '2025-10-05 00:00:00'
date_pattern = re.compile(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$')

# Function to check if a value is a date string
def is_date(value):
    if not isinstance(value, str):
        return False
    return bool(date_pattern.match(value))

# Function to process each cell
def process_cell(value):
    if pd.isna(value):
        return value

    # Convert to string for checking
    str_value = str(value)

    # Check if it's a date format
    if is_date(str_value):
        return '-'

    # Return original value if not a date
    return value

# Apply the function to the problematic column
column_name = "LR.RESULT_VALUE||(CASEWHENDBMS_LOB.GETLENGTH(LRP.RESULTVALUE)>3000THEN'/***********DATATRUNCATED***********/'||TO_CHAR(DBMS_LOB.SUBSTR(LRP.RESULTVALUE,3000,1))||'/***********DATATRUNCATED***********/'ELSETO_CHAR(LRP.RESULTVALUE)END)||(CASEWHENLR.RESUL"

# Process all columns in case the column name is different or there are other columns with dates
for column in df.columns:
    df[column] = df[column].apply(process_cell)

# Save the processed dataframe to a new Excel file
df.to_excel(output_file, index=False)

print(f"Processed file saved as {output_file}")

<ipython-input-4-f091a1f2e3a8>:12: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processed file saved as cleaned_data_1.xlsx


**Step 2**

In [5]:
# Remove all the rows which have any alphabets, '/', '-', '+', '<', '>', or ':' in the mentioned column

import pandas as pd

# Load the Excel file with the second row as header
file_path = 'cleaned_data_1.xlsx'  # Replace with your actual file path
df = pd.read_excel(file_path)  # Set header=1 to use the second row as column names

# Specify the column to clean
column_to_clean = "LR.RESULT_VALUE||(CASEWHENDBMS_LOB.GETLENGTH(LRP.RESULTVALUE)>3000THEN'/***********DATATRUNCATED***********/'||TO_CHAR(DBMS_LOB.SUBSTR(LRP.RESULTVALUE,3000,1))||'/***********DATATRUNCATED***********/'ELSETO_CHAR(LRP.RESULTVALUE)END)||(CASEWHENLR.RESUL"

# Step 1: Remove rows where the column contains blank spaces or any alphabetic characters
df = df[~df[column_to_clean].str.contains(r'[a-zA-Z\s]', na=False)]

# Step 2: Remove rows where the specified column is empty or NaN
df = df[df[column_to_clean].notna() & (df[column_to_clean].str.strip() != '')]

# Step 3: Remove rows where the column contains '/', '-', '+', '<', '>', or ':'
df = df[~df[column_to_clean].str.contains(r'[\/\-\+<>:]', na=False)]

# Save the cleaned data to a new Excel file
output_file_path = 'cleaned_data_final.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Cleaned data saved to {output_file_path}")

Cleaned data saved to cleaned_data_final.xlsx


**Step 3**

In [6]:
# Make it such that onr unique id is in one row only and rest of the thing

import pandas as pd

# Read the Excel file with headers from the second row
df = pd.read_excel('cleaned_data_final.xlsx')

# Now proceed with your data processing
print(df.columns.tolist())  # Verify the column names

# Define the id columns based on your data structure
id_columns = ['MRNO', 'PM.PREFIX||\'\'||P.PATIENTNAME', 'AGE', 'LOOKUPVALUE', 'VISITID', 'PATIENT_VISIT_ID', 'ADMISSIONNUMBER']

# Group by the identifier columns and aggregate the rest
df_grouped = df.groupby(id_columns, as_index=False).agg(lambda x: ' | '.join(x.dropna().astype(str)))

# Save the result to a new CSV file
df_grouped.to_csv('output_fixed_data.csv', index=False)

['MRNO', "PM.PREFIX||''||P.PATIENTNAME", 'AGE', 'LOOKUPVALUE', 'VISITID', 'PATIENT_VISIT_ID', 'ADMISSIONNUMBER', 'LAB_ORDER_ID', 'GROUPID', 'BILLING_GROUP', 'GROUPNAME', 'DEPARTMENT_ID', 'DEPARTMENT_CODE', 'DEPARTMENT_NAME', 'EMPLOYEE_ID', 'EMPNO', "EMPPM.PREFIX||''||EMP.FIRSTNAME||''||EMP.MIDDLENAME||''||EMP.LASTNAME", 'PROFILE_CODE', 'PROFILE_NAME', 'SERVICE_MASTER_ID', 'SERVICE_CODE', 'SERVICE_NAME', 'INVESTIGATIONPARAMETERNAME', "LR.RESULT_VALUE||(CASEWHENDBMS_LOB.GETLENGTH(LRP.RESULTVALUE)>3000THEN'/***********DATATRUNCATED***********/'||TO_CHAR(DBMS_LOB.SUBSTR(LRP.RESULTVALUE,3000,1))||'/***********DATATRUNCATED***********/'ELSETO_CHAR(LRP.RESULTVALUE)END)||(CASEWHENLR.RESUL", '(CASEWHENLR.RESULT_TYPE_ID=1099THENTO_CHAR(LRP.RESULTRANGE)ELSELR.REFERENCE_RANGEEND)', "TO_CHAR(SO.PROCESSED_DATE_TIME,'YYYY/MM/DD')", "TO_CHAR(LO.ORDERED_DATE,'YYYY/MM/DD')"]


**Step 4**

In [7]:
# Remove all the non-useful cols

import pandas as pd

# Load the CSV file
# Replace 'input_file.csv' with the actual path to your CSV file
file_path = 'output_fixed_data.csv'  # Update this with your file name if needed
df = pd.read_csv(file_path)

# Define the columns to remove
columns_to_remove = [
    "PM.PREFIX||''||P.PATIENTNAME", "VISITID", "PATIENT_VISIT_ID", "ADMISSIONNUMBER",
    "LAB_ORDER_ID", "GROUPID", "BILLING_GROUP", "GROUPNAME", "DEPARTMENT_ID",
    "DEPARTMENT_CODE", "DEPARTMENT_NAME", "EMPLOYEE_ID", "EMPNO",
    "EMPPM.PREFIX||''||EMP.FIRSTNAME||''||EMP.MIDDLENAME||''||EMP.LASTNAME",
    "PROFILE_CODE", "PROFILE_NAME", "SERVICE_MASTER_ID", "SERVICE_CODE", "SERVICE_NAME", "(CASEWHENLR.RESULT_TYPE_ID=1099THENTO_CHAR(LRP.RESULTRANGE)ELSELR.REFERENCE_RANGEEND)",
    "TO_CHAR(SO.PROCESSED_DATE_TIME,'YYYY/MM/DD')",
    "TO_CHAR(LO.ORDERED_DATE,'YYYY/MM/DD')"
]

# Remove the specified columns
df_cleaned = df.drop(columns=columns_to_remove, errors='ignore')

# Save the cleaned dataframe to a new CSV file
output_file_path = 'filtered_data.csv'
df_cleaned.to_csv(output_file_path, index=False)

print(f"Cleaned data saved to {output_file_path}")


Cleaned data saved to filtered_data.csv


**Step 5**

In [8]:
# Make each Parameter an individual column and fill the corresponding value for each row in that column

import pandas as pd

# Load the CSV file
file_path = 'filtered_data.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Specify the columns to work with
parameters_column = "INVESTIGATIONPARAMETERNAME"
values_column = "LR.RESULT_VALUE||(CASEWHENDBMS_LOB.GETLENGTH(LRP.RESULTVALUE)>3000THEN'/***********DATATRUNCATED***********/'||TO_CHAR(DBMS_LOB.SUBSTR(LRP.RESULTVALUE,3000,1))||'/***********DATATRUNCATED***********/'ELSETO_CHAR(LRP.RESULTVALUE)END)||(CASEWHENLR.RESUL"

# Ensure all values in the columns are strings or lists; replace non-iterables with empty lists
df[parameters_column] = df[parameters_column].apply(lambda x: str(x).split(" | ") if isinstance(x, str) else [])
df[values_column] = df[values_column].apply(lambda x: str(x).split(" | ") if isinstance(x, str) else [])

# Create a dictionary to hold parameter-value pairs for each row
expanded_data = []

for _, row in df.iterrows():
    param_values = dict(zip(row[parameters_column], row[values_column]))
    expanded_data.append(param_values)

# Convert the expanded data into a DataFrame
expanded_df = pd.DataFrame(expanded_data)

# Concatenate the original DataFrame with the expanded DataFrame
result_df = pd.concat([df.drop(columns=[parameters_column, values_column]), expanded_df], axis=1)

# Save the transformed DataFrame to a new CSV file
output_file_path = 'transformed_data.csv'
result_df.to_csv(output_file_path, index=False)

print(f"Transformed data saved to {output_file_path}")



Transformed data saved to transformed_data.csv
